In [ ]:
from pypowsybl_jupyter import display_sld
import ipywidgets as widgets
import pypowsybl as pp

In [ ]:
network = pp.network.create_four_substations_node_breaker_network()

In [ ]:
#declare some example callbacks, to simply log the events

out1 = widgets.Output()
def print_infos(data):   
    with out1:
        print(data)
print_infos('Events log:')


def test_nextvl_callback(event):
        print_infos('Clicked a VL arrow, next VL is: ' + event.clicked_nextvl)
def test_switch_callback(event):
        print_infos('Clicked a switch: ' + str(event.clicked_switch))
def test_feeder_callback(event):
        print_infos('Clicked a feeder: ' + str(event.clicked_feeder))  
def test_bus_callback(event):
        print_infos('Clicked a bus: ' + str(event.clicked_bus))  

In [ ]:
#demonstrates how-to create on-hover info on supported equipments, in the NAD
def format_to_html_table(row, id, type):
    table = (
        row.to_frame()
        .style.set_caption(f"{type}: {id}")
        .set_table_styles(
            [
                {
                    "selector": "caption",
                    "props": "caption-side: top; font-weight: bold; background-color: #f8f8f8; border-bottom: 1px solid #ddd; width: fit-content; white-space: nowrap;",
                },
                {
                    "selector": "th",
                    "props": "text-align: left; font-weight: bold; background-color: #f8f8f8;",
                },
                {
                    "selector": "td",
                    "props": "text-align: left;",
                },
            ]
        )
        .format(precision=3, thousands=".", decimal=",")
        .set_table_attributes('border="0"')
        .hide(axis="columns")
        .to_html()
    )
    return table
def get_equipment_info(id, type):
    if type == 'LINE':
        return format_to_html_table(network.get_lines().loc[id], id, type)
    elif type in [ 'PHASE_SHIFT_TRANSFORMER', 'TWO_WINDINGS_TRANSFORMER']:
        return format_to_html_table(network.get_2_windings_transformers().loc[id], id, type)
    elif type == 'LOAD':
        return format_to_html_table(network.get_loads().loc[id], id, type)
    elif type == 'GENERATOR':
        return format_to_html_table(network.get_generators().loc[id], id, type)
    elif type in [ 'CAPACITOR', 'INDUCTOR' ]:
        return format_to_html_table(network.get_shunt_compensators().loc[id], id, type)
    elif type in [ 'THREE_WINDINGS_TRANSFORMER', 'THREE_WINDINGS_TRANSFORMER_LEG']:
        return format_to_html_table(network.get_3_windings_transformers().loc[id], id, type)
    elif type == 'STATIC_VAR_COMPENSATOR':
        return format_to_html_table(network.get_static_var_compensators().loc[id], id, type)
    elif type in [ 'DISCONNECTOR', 'BREAKER', 'LOAD_BREAK_SWITCH', 'GROUND_DISCONNECTION' ]:
        return format_to_html_table(network.get_switches().loc[id], id, type)
    elif type == 'TIE_LINE':
        return format_to_html_table(network.get_tie_lines().loc[id], id, type)
    elif type == 'DANGLING_LINE':
        return format_to_html_table(network.get_dangling_lines().loc[id], id, type)
    elif type in [ 'LCC_CONVERTER_STATION', 'VSC_CONVERTER_STATION' ]:
        return format_to_html_table(network.get_hvdc_lines().loc[id], id, 'HDVC_LINE (' + type + ')')
    elif type == 'HVDC_LINE':
        return format_to_html_table(network.get_hvdc_lines().loc[id], id, type)
    elif type == 'BATTERY':
        return format_to_html_table(network.get_batteries().loc[id], id, type)
    elif type == 'GROUND':
        return format_to_html_table(network.get_grounds().loc[id], id, type)
    elif type == 'BUSBAR_SECTION':
        bsections=network.get_busbar_sections()
        if not bsections.empty and id in bsections.index:
            return format_to_html_table(bsections.loc[id], id, type)    
        else:
            bbvb=network.get_bus_breaker_view_buses()
            if not bbvb.empty and id in bbvb.index:
                return format_to_html_table(bbvb.loc[id], id, f'{type} (bus breaker view)')
    return f"Equipment of type '{type}' with id '{id}'"  

In [ ]:
#create an SLD widget for a network's VL and activate the callbacks on VL arrows, switches and feeders
sld_widget=display_sld(network.get_single_line_diagram(network.get_voltage_levels().index[1]), enable_callbacks=True, invalid_lf=False,
                       on_hover_func=get_equipment_info)
sld_widget.on_nextvl(test_nextvl_callback)
sld_widget.on_switch(test_switch_callback)
sld_widget.on_feeder(test_feeder_callback)
sld_widget.on_bus(test_bus_callback)

#define a text box, using one of the standard ipywidgets
sld_nextvl_text = widgets.Label()

#link the text box content to the widget's clicked_nextvl value, so that its content is automatically updated
#when a VL arrow is clicked
mylink = widgets.jslink((sld_widget, 'clicked_nextvl'), (sld_nextvl_text, 'value'))

#finally, display the widgets
widgets.VBox([widgets.HBox([widgets.Label(value='Next Voltage Level:'), sld_nextvl_text]), widgets.HBox([sld_widget, out1])])